## Import packages

In [3]:
import numpy as np
import pandas as pd
import h5py
from tqdm import tqdm

##  Read file

In [32]:
f = h5py.File('./ELMo/outputs/train_output148.ly-1.hdf5', 'r')

for k in f.keys():
    print (k)

#print (np.array(f['武威	的	公交	卡	古浪	能用']).dtype)

OSError: Unable to open file (bad object header version number)

## <font color=red>测试集test A 有两条有误！

289730只有一个选项，无法确定。

289334只有两个选项，不能or无法确定。选不能

## Part II: Add features for words in passage
Add Exact match, soft align and Pos tag features. Refer to SAN model: https://arxiv.org/abs/1712.03556

Delete soft-align and pos tag. Add option exact match.

In [10]:
maxlen_p = 150
maxlen_q = 15

In [11]:
train_path = './data/train.tsv' # train set
valid_path = './data/valid.tsv' # validation set
test_path = './data/test.tsv' # test set

In [12]:
train = pd.read_csv(train_path, sep='\t', header=0)
valid = pd.read_csv(valid_path, sep='\t', header=0)
test = pd.read_csv(test_path, sep='\t', header=0)

In [ ]:
pl = []
ql = []
for i in tqdm(range(3000)):
    line = test.iloc[i]
    passage = line['passage'].split()
    passage = '\t'.join(passage)
    passage = np.array(f[passage])
    query = line['query'].split()
    query = '\t'.join(query)
    query = np.array(f[query])
    d = np.shape(query)[1]
    
    if np.shape(passage)[0] > maxlen_p: # truncate pre
        lt = np.shape(passage)[0] - maxlen_p
        passage = passage[lt:, :]
    elif np.shape(passage)[0] < maxlen_p: # pad pre
        lt = maxlen_p - np.shape(passage)[0]
        pad = np.zeros(shape=(lt, d))
        passage = np.concatenate((pad, passage), axis=0)
    if np.shape(query)[0] > maxlen_q: # truncate post
        query = query[:maxlen_q, :]
    elif np.shape(query)[0] < maxlen_q: # pad post
        lt = maxlen_q - np.shape(query)[0]
        pad = np.zeros(shape=(lt, d))
        query = np.concatenate((query, pad), axis=0)

    ql.append(query)
    pl.append(passage)
pl = np.asarray(pl)
ql = np.asarray(ql)
print (np.shape(pl), np.shape(ql))
np.save('./data/test_elmo_p', pl)
np.save('./data/test_elmo_q', ql)